## Load Customer Complaint Analyzer Data ##

1. {dbname}.Consumer_Complaints
2. {dbname}.Customer_360_Complaints
3. {dbname}.Customer_360_Details

In [2]:
import pandas as pd
import numpy as np
import json
from teradataml import *

from collections import OrderedDict
from teradatasqlalchemy.types import *

# load vars json
with open('vars-vs_demo.json', 'r') as f:
    session_vars = json.load(f)

name = session_vars['hierarchy']['SYSDBA']['username']
pwd = session_vars['hierarchy']['SYSDBA']['password']
host = session_vars['environment']['host']

eng = create_context(host = host, username = name, password = pwd)


database = 'demo'

In [30]:
tname = 'Consumer_Complaints'
df = pd.read_csv(f'Data/{tname}.csv')

df['date_received'] = pd.to_datetime(df['date_received'])
df['date_sent_to_company'] = pd.to_datetime(df['date_sent_to_company'])
for i in df.columns:
    if df[i].dtype == 'O':
        df[i] = df[i].astype(str)

try:
    execute_sql(f'DROP TABLE {database}.{tname};')
except Exception as e:
    # Table already exists
    if str(e.args).find("3807") >= 1:
        pass
    else:
        raise

qry = f'''
CREATE MULTISET TABLE {database}.{tname} ( 
    date_received DATE FORMAT 'mm-dd-yyyy', 
    product VARCHAR(76) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    sub_product VARCHAR(42) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    issue VARCHAR(100) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    sub_issue VARCHAR(85) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    consumer_complaint_narrative VARCHAR(1100) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    company_public_response VARCHAR(119) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    company VARCHAR(46) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    state VARCHAR(40) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    zip_code VARCHAR(5) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    tags VARCHAR(29) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    consumer_consent_provided VARCHAR(20) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    submitted_via VARCHAR(11) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    date_sent_to_company DATE FORMAT 'mm/dd/yyyy', 
    company_response_to_consumer VARCHAR(31) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    timely_response VARCHAR(3) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    consumer_disputed VARCHAR(5) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    complaint_id INTEGER) NO PRIMARY INDEX;'''

execute_sql(qry)
copy_to_sql(df, table_name = tname, schema_name=database, if_exists = 'append')

In [31]:
tname = 'Customer_360_Complaints'
df = pd.read_csv(f'Data/{tname}.csv')

df['date_received'] = pd.to_datetime(df['date_received'])
df['date_sent_to_company'] = pd.to_datetime(df['date_sent_to_company'])
for i in df.columns:
    if df[i].dtype == 'O':
        df[i] = df[i].astype(str)

try:
    execute_sql(f'DROP TABLE {database}.{tname};')
except Exception as e:
    # Table already exists
    if str(e.args).find("3807") >= 1:
        pass
    else:
        raise

qry = f'''
CREATE MULTISET TABLE {database}.{tname} ( 
    date_received DATE FORMAT 'YY/MM/DD', 
    product VARCHAR(76) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    sub_product VARCHAR(42) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    issue VARCHAR(100) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    sub_issue VARCHAR(85) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    consumer_complaint_narrative VARCHAR(1100) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    company_public_response VARCHAR(119) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    company VARCHAR(46) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    state VARCHAR(40) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    zip_code VARCHAR(5) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    tags VARCHAR(29) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    consumer_consent_provided VARCHAR(20) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    submitted_via VARCHAR(11) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    date_sent_to_company DATE FORMAT 'YY/MM/DD', 
    company_response_to_consumer VARCHAR(31) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    timely_response VARCHAR(3) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    consumer_disputed VARCHAR(5) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    complaint_id INTEGER, 
    Customer_ID BIGINT) NO PRIMARY INDEX;'''

execute_sql(qry)
copy_to_sql(df, table_name = tname, schema_name=database, if_exists = 'append')

In [32]:
tname = 'Customer_360_Details'
df = pd.read_csv(f'Data/{tname}.csv')

for i in df.columns:
    if df[i].dtype == 'O':
        df[i] = df[i].astype(str)

try:
    execute_sql(f'DROP TABLE {database}.{tname};')
except Exception as e:
    # Table already exists
    if str(e.args).find("3807") >= 1:
        pass
    else:
        raise

qry = f'''
CREATE MULTISET TABLE {database}.{tname} ( 
    "Customer Identifier" BIGINT, 
    Name VARCHAR(20) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    City VARCHAR(20) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    State VARCHAR(5) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    "Customer Type" VARCHAR(20) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    "Product Holdings" VARCHAR(35) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    "Total Deposit Balance" BIGINT, 
    "Total Credit Balance" BIGINT, 
    "Total Investments AUM" BIGINT, 
    "Customer Profitability" BIGINT, 
    "Customer Lifetime Value" BIGINT, 
    "Bank Tenure" FLOAT, 
    "Affluence Segment" VARCHAR(30) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    "Digital Banking Segment" VARCHAR(20) CHARACTER SET UNICODE NOT CASESPECIFIC, 
    "Branch Banking Segment" VARCHAR(15) CHARACTER SET UNICODE NOT CASESPECIFIC) 
    PRIMARY INDEX ( "Customer Identifier" );'''

execute_sql(qry)
copy_to_sql(df, table_name = tname, schema_name=database, if_exists = 'append')

In [ ]:
remove_context()